In [1]:
import re
import typing as t

In [2]:
class TableCsv:
    def __init__(self,
                 columns: t.List[str],
                 values: t.List[t.List[t.Any]]):
        self.columns = columns
        self.values = values

def remove_commas_in_quotes(line):
    return re.sub(r'"[^"]*"', lambda m: m.group(0).replace(',', ''), line)

class Table:
    def __init__(self,
                 table_name: str,
                 csv_location: str,
                 attributes: t.Dict[str, t.Any],
                 ):
        self.table_name = table_name
        self.csv_location = csv_location
        self.attributes = attributes

    @staticmethod
    def parse_table(csv_path: str) -> 'Table':
        with open(csv_path, 'r') as f:
            raw_contents = f.readlines()
        table_name = raw_contents[0].replace(',', '').strip()
        attributes: t.Dict[str, t.Any] = {}
        current_block = []
        for line in raw_contents[1:]:
            cleaned_line = remove_commas_in_quotes(line)
            cleaned_line = cleaned_line.replace('"', '').strip()
            if cleaned_line.replace(',', '').strip() == '':
                # start of the next attribute
                if len(current_block) <= 0:
                    continue
                sub_table_name = [v.strip() for v in current_block[0].split(',') if v.strip() != '']
                if len(sub_table_name) == 1 and len(current_block) >= 3:
                    # treat it as a csv table.
                    sub_table_name = sub_table_name[0]
                    columns = [col.strip() for col in current_block[1].split(',')]
                    values = []
                    for l in current_block[2:]:
                        value = [val.strip() for val in l.split(',')]
                        values.append(value)
                    attributes[sub_table_name] = TableCsv(columns, values)
                else:
                    # treat it an unorganized texts
                    for l in current_block:
                        attr_name = l.split(',')[0]
                        attr_content = '|'.join([v.strip() for v in l.split(',')[1:] if v.strip() != ''])
                        attributes[attr_name] = attr_content
                current_block = []
            else:
                current_block.append(cleaned_line)
        return Table(table_name, csv_path, attributes)

In [3]:
table = Table.parse_table("../../input/csn-data-book-2024-csv/CSVs/2024_CSN_Fraud_Reports_by_Amount_Lost.csv")

In [4]:
def solve_q1(table: Table) -> float:
    total_reports = int(table.attributes['Reports with $ Loss'].split('|')[0])
    sub_table = table.attributes['Reported Fraud Losses in $1 - $1000 Range']
    total_below_500 = 0
    for row in sub_table.values:
        if row[0] in ['$1 - $100', '$101 - $200', '$201 - $300', '$301 - $400', '$401 - $500']:
            total_below_500 += int(row[1])
    return total_below_500/total_reports
solve_q1(table)

0.5228329552819183